# 创建候选区块

每个“矿工”都有一个属于自己的候选区块，最终只有获胜“矿工”的候选区块会被写进区块链得到“续命”，而其他候选区块的生命在“矿工”验证获胜“矿工”广播的区块有效起得到“终结”。当候选区块创建好后，“矿工”开始“挖矿”，即通过枚举nonce的方式求出满足条件的工作量证明nonce值。

根据区块高度计算区块奖励的函数示例如下：

In [1]:
def get_block_reward(height, fees=0):
    # 减半间隔为210000个区块
    reward_interval = 210000
    # 初始建立50比特币
    reward = 50
    # 减半次数
    halvings = height // reward_interval

    # 如果减半次数超过64，返回交易费
    if halvings >= 64:
        return fees

    # 否则奖励减半
    reward >>= halvings
    return reward + fees

### 创币交易

In [2]:
from simchain import Tx
# 创币交易，地址'bb'，金额为5000分
tx = Tx.create_coinbase('bb', 5000)
tx

Tx(id:f6d6ed236316ca82747ef4e04b23cad3315ce43e74a0ef5b79a4c8acace1c5ba)

In [3]:
# 只有一个输出单元
tx.tx_out

[Vout(to_addr:bb,value:5000)]

In [4]:
# 只有一个输入单元，且定位地址为None
tx.tx_in

[Vin(to_spend:None,signature:b'\xbc4\xbf\x90\xcf.\xfb\xde\xa0\x0bZ\xbd\x08=\xf6\xaevwE\xfb\xeaN4\xc8p8\xbd~"\x02#&',pubkey:None)]

In [5]:
tx.is_coinbase

True

### 定义创建候选区块的函数

In [6]:
# 输入参数为选择要打包的交易列表，矿工收款地址、前区块、前区块的高度和本地时间
def create_candidate_block(self):
    self.choose_tx_candidates() 
    txs = self.candidate_block_txs
    # 计算总奖励，为“挖矿”奖励和交易费之和
    value = self.get_block_reward() + self.calculate_fees(txs)
    # 创建创币交易
    coinbase = self.create_coinbase(value)
    # 将创币交易插入到交易列表第1条
    txs = [coinbase] + txs

    # 获取前区块的哈希值
    prev_block_hash = self.blockchain[-1].hash
    bits = Params.INITIAL_DIFFICULTY_BITS
    # 创建候选区块
    self.candidate_block = Block(version=0,
                                 prev_block_hash=prev_block_hash,
                                 timestamp=self.network.time[-1],
                                 bits=bits,
                                 nonce=0,
                                 txs=txs or [])

    self._is_block_candidate_created = True

In [7]:
from simchain import Network
net = Network()
zhangsan = net.peers[0]
net.make_random_transactions()
# 张三创建候选区块
zhangsan.create_candidate_block()
# 访问张三的候选区块
zhangsan.candidate_block

2019-06-13 23:51:45,591 - A blockchain p2p network created,12 peers joined
2019-06-13 23:51:45,627 - genesis block has been generated
2019-06-13 23:51:45,670 - peer(52, 99)(pid=3) created a transaction
2019-06-13 23:51:45,671 - peer(52, 99)(pid=3) sent a transaction to network
2019-06-13 23:51:46,503 - peer(52, 99)(pid=3)'s transaction verified by 11 peers
2019-06-13 23:51:46,542 - peer(70, 9)(pid=1) created a transaction
2019-06-13 23:51:46,543 - peer(70, 9)(pid=1) sent a transaction to network
2019-06-13 23:51:47,380 - peer(70, 9)(pid=1)'s transaction verified by 11 peers
2019-06-13 23:51:47,422 - peer(28, 13)(pid=4) created a transaction
2019-06-13 23:51:47,425 - peer(28, 13)(pid=4) sent a transaction to network
2019-06-13 23:51:48,279 - peer(28, 13)(pid=4)'s transaction verified by 11 peers


Block(hash:38175dcb4e784d1d19c66f80ffd081f06e5725e423126fa2b4583230570de075)

In [8]:
# 候选区块中的交易
zhangsan.candidate_block.txs

[Tx(id:e3b9bfc01aa9c047eb727a041140e0d2146f44bf10dfe69d3a5cef95a1db6488),
 Tx(id:0e8ecbd047af6617e386c108eb40b81e16ef731461662cf3fb4e7f93a2c10579),
 Tx(id:5c825b99e249628a980ef0615d679ddd3be65bdb7bee5610972afd526f915022),
 Tx(id:4e8909c8bce66b260e4e682161dc66000bee0cd4fb4c603149d34178e6d82321)]

In [9]:
# 获取交易列表第1条交易
tx = zhangsan.candidate_block.txs[0]
# 创币交易
tx.is_coinbase

True

In [10]:
# 金额为530
tx.tx_out[0].value

530

In [11]:
# 打包交易为4条，每条交易费固定为10分
len(zhangsan.candidate_block.txs)

4

In [12]:
# 改变交易选择策略为“随机”选择
zhangsan.tx_choice_method = 'random'
zhangsan.create_candidate_block()
len(zhangsan.candidate_block.txs)

4

# 挖矿、打包、广播区块

## 挖矿

In [13]:
from simchain.consensus import mine
# 计算满足条件的nonce
nonce = mine(zhangsan.candidate_block)
nonce

423371

In [14]:
# 也可以直接调用节点对象的consensus()方法
zhangsan.consensus()

423371

## 打包候选区块

In [15]:
# 节点打包区块
block = zhangsan.package_block(nonce)
block

Block(hash:000023d61f7c0aa69b7c4a6e8c9df6d294d39c2225240d6123d56907c476e14f)

In [16]:
# 替换nonce
block1 = zhangsan.candidate_block._replace(nonce=nonce)
block1

Block(hash:000023d61f7c0aa69b7c4a6e8c9df6d294d39c2225240d6123d56907c476e14f)

In [17]:
from simchain.consensus import caculate_target
# nonce满足条件
int(block.hash, 16) < caculate_target(block.bits)

True

In [18]:
from simchain import Block
block.version

0

In [19]:
# 仅改变之前区块的版本号
block2 = Block(2, block.prev_block_hash, block.timestamp, block.bits, nonce, block.txs)
block2

Block(hash:0a1ed8398c2fa6ab2eb4266ef9938f7618e9bb79627c73cb6ec70e2343acf568)

In [20]:
# nonce不满足条件
int(block2.hash, 16) < caculate_target(block2.bits)

False

由上述可知，候选区块的创建是一次性的，创建好后，除了nonce值的替换外，其他数据都不能修改。

## 广播区块

和广播交易类似，当获胜“矿工”成功打包区块后，会将区块广播到网络中，通过泛洪路由协议传播到其他节点。由于验证一个区块比验证一个交易复杂得多，且数据量较单笔交易大得多，导致区块广播比交易广播耗时大很多。